In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install nflows --quiet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import savefig
from scipy.io import arff
import ntpath
import glob
import os
import math
from sklearn import preprocessing
# !pip install liac-arff
#import arff
import argparse

import torch
from torch import nn, optim
from nflows.flows import Flow
from nflows.distributions import StandardNormal
from nflows.transforms import CompositeTransform, MaskedAffineAutoregressiveTransform, RandomPermutation
from nflows.transforms import AffineCouplingTransform

from sklearn import manifold
import string

import pickle


In [ ]:

# Load object from file
with open(gdrivePath +os.sep + "data" +os.sep+'X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)
with open(gdrivePath +os.sep + "data" +os.sep+'X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)
with open(gdrivePath +os.sep + "data" +os.sep+'y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)
with open(gdrivePath +os.sep + "data" +os.sep+'y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)


In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
unique_values, counts = np.unique(y_train, return_counts=True)
display(dict(zip(unique_values, counts)),np.max(counts))

In [ ]:
def FB_Oversampler(num_synthetic_samples, X_min,num_layers,hidden_features,learning_rate):


  data = torch.tensor(X_min, dtype=torch.float32)

  # Define the number of features and hidden features
  num_features = X_min.shape[1]
  #hidden_features = 128  # number of hidden units in the neural network

  # a sequence of masked affine autoregressive transforms
  #num_layers = 2

  # Base distribution: standard normal
  base_distribution = StandardNormal([num_features])

  # the sequence of transformations
  transforms = []
  for _ in range(num_layers):
      transforms.append(MaskedAffineAutoregressiveTransform(features=num_features, hidden_features=hidden_features)) #MaskedAffineAutoregressiveTransform(features=num_features, hidden_features=hidden_features)
      transforms.append(RandomPermutation(features=num_features))
  transform = CompositeTransform(transforms)

  #  the flow model
  flow = Flow(transform, base_distribution)

  # Train the model
  optimizer = optim.Adam(flow.parameters(), lr=learning_rate)
  num_epochs = 50
  batch_size = 128


  data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)

  for epoch in range(num_epochs):
      for batch in data_loader:
          optimizer.zero_grad()
          loss = -flow.log_prob(batch).mean()
          loss.backward()
          optimizer.step()
      #print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

  # Generate new samples
  with torch.no_grad():
      samples = flow.sample(num_synthetic_samples)
  generated_data_np = samples.numpy()

  return generated_data_np

In [ ]:
# Get unique values and their counts
unique_values, counts = np.unique(y_train, return_counts=True)
classlabel_counts = dict(zip(unique_values, counts))
max_count = 759

i=1
for label, count in classlabel_counts.items():
    #print(x.shape, y.shape)
    #print("label, count, max_count",label,count,max_count)
    X_minority = X_train[y_train == label]
    #X_ori_min = X_train[y_train == label]
    #print("Real",X_minority.min(), X_minority.max())
    if count < max_count:
        #print("\n")
        print(f"Value {label} appears {count} times.")
        num_synthetic_samples = max_count - count
        synthetic_samples = FB_Oversampler(int(num_synthetic_samples), X_minority, 2, 64, 1e-6) #num_layers,hidden_features,learning_rate
        if i==1:
          X_train_gen = X_minority
          X_train_gen = np.vstack([X_train_gen, synthetic_samples])

          y_train_gen = np.full(759, label)

          y_train_indexes = np.full(count, 1)
          y_train_indexes = np.concatenate([y_train_indexes, np.full(num_synthetic_samples, 2)])
        else:
          tmp = np.vstack([X_minority, synthetic_samples])
          X_train_gen = np.vstack([X_train_gen, tmp])
          y_train_gen = np.concatenate([y_train_gen, np.full(759, label)])

          y_train_indexes = np.concatenate([y_train_indexes, np.full(count, 1)])
          y_train_indexes = np.concatenate([y_train_indexes, np.full(num_synthetic_samples, 2)])
    else:
        X_train_gen = np.vstack([X_train_gen, X_minority])
        y_train_gen = np.concatenate([y_train_gen, np.full(count, label)])
        y_train_indexes = np.concatenate([y_train_indexes, np.full(count, 1)])
    i=i+1

In [ ]:
import pickle

# Save object to file
with open(gdrivePath +os.sep+"results"+os.sep+"FB"+os.sep+'FB_X_train.pkl', 'wb') as f:
    pickle.dump(X_train_gen, f)

with open(gdrivePath +os.sep+"results"+os.sep+"FB"+os.sep+'FB_X_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)

with open(gdrivePath +os.sep+"results"+os.sep+"FB"+os.sep+'FB_y_train.pkl', 'wb') as f:
    pickle.dump(y_train_gen, f)

with open(gdrivePath +os.sep+"results"+os.sep+"FB"+os.sep+'FB_y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)

with open(gdrivePath +os.sep+"results"+os.sep+"FB"+os.sep+'FB_y_train_idx.pkl', 'wb') as f:
    pickle.dump(y_train_indexes, f)